# REDUCEDHEATCARB simulation

This JupyterLabs notebook can be used for physics ifnormed machine learning analysis in the REDUCEDHEATCAB project.
Don't forget to install the requirements listed in [requirements.txt](../requirements.txt) first!

## Setting the stage

First several imports and variables need to be defined


### Imports and generic settings

In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib widget


from tqdm.notebook import tqdm

from gekko import GEKKO

import sys
sys.path.append('../data/')
sys.path.append('../view/')
sys.path.append('../analysis/')
from nfh_utils import *

# usually, two decimals suffice for displaying DataFrames (NB internally, precision may be higher)
pd.options.display.precision = 2

%load_ext autoreload

from preprocessor import Preprocessor
from rhc_analysis import Learner

%matplotlib inline
%matplotlib widget

from plotter import Plot



### Reading preprocessed interpolated properties from a parquet file

In [ ]:
%%time

rhc_preprocessed_poperties_file='rhc_preprocessed_properties.parquet'

# Attempt to read the Parquet file
try:
    df_prep = pd.read_parquet(
        rhc_preprocessed_poperties_file, 
        engine='pyarrow',
        dtype_backend='numpy_nullable'
        )
    print("File was successfully read without specifying compression codec.")
except Exception as e:
    print(f"Error reading file: {e}")

In [ ]:
#sorting the DataFrame index is needed to get good performance on certain filters
#this guarding code to check whether DataFrames are properly sorted
if not df_prep.index.is_monotonic_increasing:
    print('df needed index sorting')
    df_prep = df_prep.sort_index()  

In [ ]:
print(f"df_prep.count().sum(): {df_prep.count().sum():_}")

In [ ]:
df_prep.info()

In [ ]:
print("[\n'","', \n'".join(sorted(df_prep.columns)),"'\n]")

In [ ]:
%autoreload 2
units_to_mathtext = property_types = {
    'degC' : r'$°C$',
    'ppm' : r'$ppm$',
    '0' : r'$[-]$',
    'bool': r'$0 = False; 1 = True$',
    'p' : r'$persons$',
    'W' : r'$W$',
    'W_m_2' : r'$W/m^{2}$',
    'm_s_1' : r'$m/s$'    
}

### Calculate additional properties 

In [ ]:
%%time
df_prep['calculated_e_use__W'] = df_prep['device_p1-reader_e_use_hi__W'] + df_prep['device_p1-reader_e_use_lo__W']
df_prep['calculated_e_ret__W'] = df_prep['device_p1-reader_e_ret_hi__W'] + df_prep['device_p1-reader_e_ret_lo__W'] 


## Simulation

Using the [GEKKO Python](https://machinelearning.byu.edu/) dynamic optimization toolkit.

In [ ]:
#select column names
property_sources = {
    'temp_indoor__degC' : 'batch_import_remeha_temp_indoor__degC',
    'temp_outdoor__degC' : 'batch_import_KNMI_temp_outdoor__degC',
    'g_use_hhv__W' : 'device_p1-reader_g_use_hhv__W',
    'e_use__W' : 'calculated_e_use__W',
    'e_ret__W' : 'calculated_e_ret__W'
}

### Result Visualization

In [ ]:
df_plot = df_prep[[prop for prop in df_prep.columns.values if prop.split('__')[-1] == 'degC']]

In [ ]:
#Plot only temperatures from all sources for all ids
Plot.dataframe_preprocessed_plot(df_plot, units_to_mathtext)

In [ ]:
df_lot = df_prep

In [ ]:
#Plot all properties from all sources for all ids
Plot.dataframe_preprocessed_plot(df_plot, units_to_mathtext)